In [ ]:
import pandas as pd
import numpy as np
import random
from math import ceil, sqrt
from shapely.geometry import Point, Polygon
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

def svgCloudEdge(x0, y0, x1, y1, h=20, r=14, rd=10, _stroke_='#ff0000', _fill_='#ff0000'):    
    vx, vy   = x1 - x0, y1 - y0
    l        = sqrt(vx * vx + vy * vy)
    vx, vy   = vx / l, vy / l
    vxp, vyp = vy, -vx
    _svg_ = f'<path d="m {x0} {y0} l {vxp*h} {vyp*h} l {vx*l} {vy*l} l {-vxp*h} {-vyp*h} z" stroke="{_stroke_}" fill="{_fill_}" />'

    circles = ceil(l / r)
    for i in range(int(circles)+1):
        x, y = x0 + vxp*h + vx*l*i/circles, y0 + vyp*h + vy*l*i/circles
        rr, xr, yr = r + rd*(random.random()-0.5), x + rd*(random.random()-0.5), y + rd*(random.random()-0.5)
        _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'

    rr     = h/2
    xr, yr = x0 + vxp*rr, y0 + vyp*rr
    _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'

    rr     = h/2
    xr, yr = x0 + vxp*rr + l*vx, y0 + vyp*rr + l*vy
    _svg_ += f'<circle cx="{xr}" cy="{yr}" r="{rr}" stroke="{_stroke_}" fill="{_fill_}" />'
    return _svg_


In [ ]:
_svg_ =  '<svg width="300" height="300">'
_svg_ += '<rect x="0" y="0" width="300" height="300" fill="#ffffff"/>'
_svg_ += svgCloudEdge(100,  50, 200,  50)
_svg_ += svgCloudEdge(200,  50, 200, 150)
_svg_ += svgCloudEdge(200, 150, 100, 150)
_svg_ += svgCloudEdge(100, 150, 100,  50)
_svg_ += '</svg>'
rt.displaySVG(_svg_)

In [ ]:
import math

x,y,w,h = 100,100,200,200
my_rectangle = Polygon([(x,y), (x,y+h), (x+w,y+h), (x+w,y)])
my_triangle = Polygon([(x+w/2,y), (x,y+h), (x+w,y+h)])
framing_start = '<svg x="0" y="0" width="400" height="400"><rect x="0" y="0" width="500" height="500" fill="#ffffff"/>'
framing_end   = '</svg>'

def extrudePolygon(core, r): # Only works on convex shapes...
    pts     = core.exterior.coords
    extrusion_pts = []
    for i in range(0, len(pts)-1):
        x0,y0,x1,y1 = pts[i][0], pts[i][1], pts[i+1][0], pts[i+1][1]
        vx, vy      = x1-x0, y1-y0
        l           = math.sqrt(vx*vx + vy*vy)
        if l < 0.001:
            l = 1.0
        vxn, vyn     = vx/l, vy/l
        vxn_p, vyn_p = -vyn, vxn
        extrusion_pts.append((x0+vxn_p*r, y0+vyn_p*r))
        extrusion_pts.append((x1+vxn_p*r, y1+vyn_p*r))
    extrusion = Polygon(extrusion_pts)
    return extrusion

def cloudCircleEdges(core, r):
    circle_tuples = []
    pts     = core.exterior.coords
    for i in range(0, len(pts)-1):
        x0,y0,x1,y1 = pts[i][0], pts[i][1], pts[i+1][0], pts[i+1][1]
        vx, vy      = x1-x0, y1-y0
        l           = math.sqrt(vx*vx + vy*vy)
        if l < 0.001:
            l = 1.0
        vxn, vyn     = vx/l, vy/l
        vxn_p, vyn_p = -vyn, vxn

        t = random.random() * (r/2)
        while t < l:
            from_edge = random.random() * r
            radius    = random.random() * r/2 + r/2
            circle_tuples.append((x0+vxn*t+vxn_p*from_edge, y0+vyn*t+vyn_p*from_edge, radius))
            t += random.random() * (3*r/4)
    return circle_tuples

def svgCloud(core, r=20):
    svg = ''
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(core)}" fill="none" stroke="#000000"/>'
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(extrudePolygon(core,r))}" fill="none" stroke="#ff0000"/>'
    _tuples_ = cloudCircleEdges(core, r)
    _points_ = {}
    for _t_ in _tuples_:
        svg += f'<circle cx="{_t_[0]}" cy="{_t_[1]}" r="{_t_[2]}" fill="none" stroke="#000000" />'
        for _angle_ in range(0,360):
            _x_ = _t_[0] + _t_[2] * math.cos(_angle_ * pi / 180.0)
            _y_ = _t_[1] + _t_[2] * math.sin(_angle_ * pi / 180.0)
            _points_[str(_t_) + '_' + str(_angle_)] = [_x_,_y_]
    convex_hull_keys  = rt.grahamScan(_points_)
    convex_hull_pts   = []
    for _p_ in convex_hull_keys:
        convex_hull_pts.append(_points_[_p_])
    convex_hull_poly = Polygon(convex_hull_pts) 
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(convex_hull_poly)}" fill="none" stroke="#ff0000" stroke-width="2"/>'
    return svg

rt.displaySVG(rt.tile([framing_start + svgCloud(my_rectangle) + framing_end, 
                       framing_start + svgCloud(my_rectangle) + framing_end, 
                       framing_start + svgCloud(my_rectangle) + framing_end]))

In [ ]:
rt.displaySVG(rt.tile([framing_start + svgCloud(my_triangle) + framing_end, 
                       framing_start + svgCloud(my_triangle) + framing_end, 
                       framing_start + svgCloud(my_triangle) + framing_end]))

In [ ]:
def cloudCircleEdges(core, r):
    circle_tuples = []
    pts     = core.exterior.coords
    for i in range(0, len(pts)-1):
        x0,y0,x1,y1 = pts[i][0], pts[i][1], pts[i+1][0], pts[i+1][1]
        vx, vy      = x1-x0, y1-y0
        l           = math.sqrt(vx*vx + vy*vy)
        if l < 0.001:
            l = 1.0
        vxn, vyn     = vx/l, vy/l
        vxn_p, vyn_p = -vyn, vxn
        t = 0.0
        while t < l + 0.01:
            from_edge = random.random() * r
            radius    = random.random() * r/2 + r/3
            circle_tuples.append((x0+vxn*t+vxn_p*from_edge, y0+vyn*t+vyn_p*from_edge, radius))
            t += r/2
    return circle_tuples

#
# minimumDistanceFromRayToPoint() - minimum distance from a ray to a point
#
# Derived From:  https://math.stackexchange.com/questions/3894559/how-to-check-if-a-ray-intersects-a-circle
#
# ray   == (x,  y, vx, vy)
# point == (xc, yc) # circle center notation...
#
from math import sqrt
def minimumDistanceFromRayToPoint(ray, point):
    x, y, vx, vy = ray
    xc, yc = point
    lam = -((x - xc) * vx + (y - yc) * vy)/(vx**2 + vy**2)
    r_star = sqrt((x + lam * vx - xc)**2 + (y + lam * vy - yc)**2)
    return r_star

#
# Derived from:  https://stackoverflow.com/questions/1073336/circle-line-segment-collision-detection-algorithm
#
def closestCircleIntersection(x,y,vx,vy,circles):
    xi,yi,ti = None,None,None
    for circle in circles:
        xc,yc,r = circle
        d = minimumDistanceFromRayToPoint((x,y,vx,vy), (xc,yc))
        if d <= r:
            t = vx*(xc-x) + vy*(yc-y)
            ex = x + t * vx
            ey = y + t * vy
            e_to_c = sqrt((xc-ex)**2 + (yc-ey)**2)
            if e_to_c < r:
                dt = sqrt(r**2 - e_to_c**2)
                fx = (t-dt) * vx + x
                fy = (t-dt) * vy + y
                if ti is None or ti > t:
                    xi,yi,ti = fx,fy,t
                gx = (t+dt) * vx + x
                gy = (t+dt) * vy + y
                if ti is None or ti > t:
                    xi,yi,ti = gx,gy,t
            else:
                if ti is None or ti > t:
                    xi,yi,ti = ex,ey,t
    return xi,yi

def svgCloud(core, r=20):
    svg = ''
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(core)}" fill="none" stroke="#000000"/>'
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(extrudePolygon(core,r))}" fill="none" stroke="#ff0000"/>'    
    # Create circles that form the outline and convert to points
    _tuples_ = cloudCircleEdges(core, r)
    _points_ = {}
    for _t_ in _tuples_:
        svg += f'<circle cx="{_t_[0]}" cy="{_t_[1]}" r="{_t_[2]}" fill="none" stroke="#000000" />'
        for _angle_ in range(0,360,8):
            _x_ = _t_[0] + _t_[2] * math.cos(_angle_ * pi / 180.0)
            _y_ = _t_[1] + _t_[2] * math.sin(_angle_ * pi / 180.0)
            _points_[str(_t_) + '_' + str(_angle_)] = [_x_,_y_]
    # Calculate the convex hull
    convex_hull_keys  = rt.grahamScan(_points_)
    convex_hull_pts   = []
    for _p_ in convex_hull_keys:
        convex_hull_pts.append(_points_[_p_])
    #convex_hull_poly = Polygon(convex_hull_pts) 
    #svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(convex_hull_poly)}" fill="none" stroke="#ff0000" stroke-width="2"/>'
    # Shrink wrap the convex hull
    shrink_wrap_pts = []
    for i in range(0,len(convex_hull_pts)-1):
        x0, y0, x1, y1 = convex_hull_pts[i][0], convex_hull_pts[i][1], convex_hull_pts[i+1][0], convex_hull_pts[i+1][1]
        vx, vy      = x1-x0, y1-y0
        l           = math.sqrt(vx*vx + vy*vy)
        if l < 0.001:
            l = 1.0
        vxn,vyn     = vx/l, vy/l
        vxn_p,vyn_p = -vyn, vxn
        shrink_wrap_pts.append((x0,y0))
        t = 1.0
        while t < l:
            x,y = closestCircleIntersection(x0+t*vx,y1+t*vy,vxn_p,vyn_p,_tuples_)
            if x is None:
                x,y = closestCircleIntersection(x0+t*vx,y1+t*vy,-vxn_p,-vyn_p,_tuples_)
            if x is None:
                shrink_wrap_pts.append((x0+t*vx,y0+t*vy))
            else:
                shrink_wrap_pts.append((x,y))
            t += 1.0
        shrink_wrap_pts.append((x1,y1))
    svg += f'<path d="{rt.shapelyPolygonToSVGPathDescription(Polygon(shrink_wrap_pts))}" fill="none" stroke="#ff0000" stroke-width="2"/>'
    return svg

rt.displaySVG(rt.tile([framing_start + svgCloud(my_rectangle) + framing_end, 
                       framing_start + svgCloud(my_rectangle) + framing_end, 
                       framing_start + svgCloud(my_rectangle) + framing_end]))

In [ ]:
rt.displaySVG(rt.tile([framing_start + svgCloud(my_triangle) + framing_end, 
                       framing_start + svgCloud(my_triangle) + framing_end, 
                       framing_start + svgCloud(my_triangle) + framing_end]))

In [ ]:
#
# minimumDistanceFromRayToPoint() - minimum distance from a ray to a point
#
# Derived From:  https://math.stackexchange.com/questions/3894559/how-to-check-if-a-ray-intersects-a-circle
#
# ray   == (x,  y, vx, vy)
# point == (xc, yc) # circle center notation...
#
from math import sqrt
def minimumDistanceFromRayToPoint(ray, point):
    x, y, vx, vy = ray
    xc, yc = point
    lam = -((x - xc) * vx + (y - yc) * vy)/(vx**2 + vy**2)
    r_star = sqrt((x + lam * vx - xc)**2 + (y + lam * vy - yc)**2)
    return r_star

minimumDistanceFromRayToPoint((0,0,1,0), (4,4))
